In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# import hyperas
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


import tensorflow as tf


from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import STATUS_OK

In [24]:
df = pd.read_csv("model_data.csv")
df.drop("Date", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27140 entries, 0 to 27139
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        27140 non-null  int64  
 1   temp              27140 non-null  float64
 2   dewp              27140 non-null  float64
 3   slp               27140 non-null  float64
 4   visib             27140 non-null  float64
 5   wdsp              27140 non-null  float64
 6   max               27140 non-null  float64
 7   min               27140 non-null  float64
 8   fog               27140 non-null  float64
 9   rain_drizzle      27140 non-null  float64
 10  snow_ice_pellets  27140 non-null  float64
 11  hail              27140 non-null  float64
 12  thunder           27140 non-null  float64
 13  year sin          27140 non-null  float64
 14  year cos          27140 non-null  float64
dtypes: float64(14), int64(1)
memory usage: 3.1 MB


In [8]:
# Train, Validation, Test Split
train_df = df[0:int(len(df)*.7)]
val_df = df[int(len(df)*.7):int(len(df)*.9)]
test_df = df[int(len(df)*.9):]


In [9]:
scaler = StandardScaler()
scaler.fit(train_df)


array([[-1.73195964, -1.23770024, -1.2151279 , ..., -0.47000483,
        -0.03276567, -1.40870429],
       [-1.7317773 , -0.58016614, -0.46015475, ..., -0.47000483,
        -0.05707983, -1.40790556],
       [-1.73159496, -0.27488245,  0.06197622, ..., -0.47000483,
        -0.08137666, -1.40668867],
       ...,
       [ 1.73159496,  1.34546944,  0.99804885, ..., -0.47000483,
        -0.23631966,  1.39930184],
       [ 1.7317773 ,  1.27762862,  1.00980855, ..., -0.47000483,
        -0.21230455,  1.40318689],
       [ 1.73195964,  1.14716551,  0.99804885, ...,  0.52644108,
        -0.18822618,  1.40665823]])

In [10]:
train_scaled = scaler.transform(train_df)
val_scaled = scaler.transform(val_df)
test_scaled = scaler.transform(test_df)

slice(0, 30, None)

In [18]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_scaled, val_df=val_scaled, test_df=test_scaled,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [19]:
w1 = WindowGen(input_width=30, label_width=1, shift=3, label_columns=["temp"])
w1

In [29]:
# TD

def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns:
        labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

    inputs.set_shate([None, self.input_width, None])
    inputs.set_shape([None, self.lable_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

In [37]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    dset = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=128)
    

    return dset
WindowGenerator.make_dataset = make_dataset

TypeError: make_dataset() missing 1 required positional argument: 'data'

In [ ]:
model = Sequential
model.add(LSTM({{uniform(10, 100)}}, input_shape = (X_train.shape[1], X_train.shape[2])))